In [9]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKey, Boolean

In [10]:
metadata = MetaData()
engine = create_engine(
    "sqlite:///../output/zoo.db", echo=True, connect_args={"check_same_thread": False}
)

In [11]:
cookies = Table(
    "cookies",
    metadata,
    Column("cookie_id", Integer(), primary_key=True),
    Column("cookie_name", String(50), index=True),
    Column("cookie_url", String(255)),
    Column("cookie_sku", String(55)),
    Column("quantity", Integer()),
    Column("unit_cost", Numeric(12, 2)),
    extend_existing=True,
)
from datetime import datetime
from sqlalchemy import DateTime

users = Table(
    "users",
    metadata,
    Column("user_id", Integer(), primary_key=True),
    Column("username", String(15), nullable=False, unique=True),
    Column("email_address", String(255), nullable=False),
    Column("phone", String(20), nullable=False),
    Column("password", String(25), nullable=False),
    Column("create_on", DateTime(), default=datetime.now),
    Column("update_on", DateTime(), default=datetime.now, onupdate=datetime.now),
    extend_existing=True,
)
from sqlalchemy import ForeignKey, Boolean

orders = Table(
    "orders",
    metadata,
    Column("order_id", Integer(), primary_key=True),
    Column("user_id", ForeignKey("users.user_id")),
    Column("shipped", Boolean(), default=False),
    extend_existing=True,
)
line_items = Table(
    "line_items",
    metadata,
    Column("line_id", Integer(), primary_key=True),
    Column("order_id", ForeignKey("orders.order_id")),
    Column("cookie_id", ForeignKey("cookies.cookie_id")),
    Column("quantity", Integer()),
    Column("extended_cost", Numeric(12, 2)),
    extend_existing=True,
)
metadata.create_all(engine)

2025-05-16 08:43:10,255 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-16 08:43:10,255 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2025-05-16 08:43:10,256 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-16 08:43:10,257 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-05-16 08:43:10,257 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-16 08:43:10,258 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-05-16 08:43:10,258 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-16 08:43:10,259 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("line_items")
2025-05-16 08:43:10,260 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-16 08:43:10,260 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
employee_table = Table(
    "employee",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("manager", None, ForeignKey("employee.id")),
    Column("name", String(255)),
)

In [13]:
from sqlalchemy import select, func

all_orders = select(users.c.username, func.count(orders.c.order_id))
all_orders = all_orders.select_from(users.outerjoin(orders))

all_orders = all_orders.group_by(users.c.username)

cnn = engine.connect()
result = cnn.execute(all_orders).fetchall()
for row in result:
    print(row)

2025-05-16 08:43:10,512 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-16 08:43:10,513 INFO sqlalchemy.engine.Engine SELECT users.username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username
2025-05-16 08:43:10,514 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ()
('cakeeater', 0)
('cookiemon', 1)
('pieguy', 0)


In [14]:
def get_order_by_customer(cust_name):
    cust_orders = select(
        orders.c.order_id,
        users.c.username,
        users.c.phone,
        cookies.c.cookie_name,
        line_items.c.quantity,
        line_items.c.extended_cost,
    )
    cust_orders = cust_orders.select_from(
        users.join(orders).join(line_items).join(cookies)
    )
    cust_orders = cust_orders.where(users.c.username == cust_name)
    result = cnn.execute(cust_orders).fetchall()
    return result


get_order_by_customer("cakeeater")

2025-05-16 08:43:10,629 INFO sqlalchemy.engine.Engine SELECT orders.order_id, users.username, users.phone, cookies.cookie_name, line_items.quantity, line_items.extended_cost 
FROM users JOIN orders ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = ?
2025-05-16 08:43:10,629 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ('cakeeater',)


[]

In [15]:
from sqlalchemy import text

cnn.execute(text("select * from cookies")).fetchall()

2025-05-16 08:43:10,764 INFO sqlalchemy.engine.Engine select * from cookies
2025-05-16 08:43:10,764 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()


[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, 0.5),
 (2, 'peanut butter', 'http://some.aweso.me/cookie/recipe.html', 'PB01', 24, 0.25),
 (3, 'oatmeal raisin', 'http://some.aweso.me/cookie/recipe.html', 'EWW01', 100, 1),
 (6, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, 0.25),
 (7, 'oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, 1)]

In [16]:
from sqlalchemy import select

In [17]:
s = select(cookies)
rp = cnn.execute(s)
result = rp.fetchall()
result

2025-05-16 08:43:10,979 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-16 08:43:10,980 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()


[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50')),
 (2, 'peanut butter', 'http://some.aweso.me/cookie/recipe.html', 'PB01', 24, Decimal('0.25')),
 (3, 'oatmeal raisin', 'http://some.aweso.me/cookie/recipe.html', 'EWW01', 100, Decimal('1.00')),
 (6, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25')),
 (7, 'oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, Decimal('1.00'))]

In [18]:
s = cookies.select()
rp = cnn.execute(s)
result = rp.fetchall()
print(result)

2025-05-16 08:43:11,087 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-16 08:43:11,088 INFO sqlalchemy.engine.Engine [cached since 0.1093s ago] ()
[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50')), (2, 'peanut butter', 'http://some.aweso.me/cookie/recipe.html', 'PB01', 24, Decimal('0.25')), (3, 'oatmeal raisin', 'http://some.aweso.me/cookie/recipe.html', 'EWW01', 100, Decimal('1.00')), (6, 'peanut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25')), (7, 'oatmeal raisin', 'http://some.okay.me/cookie/raisin.html', 'EWW01', 100, Decimal('1.00'))]


In [19]:
firstrow = result[0]
firstrow

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50'))

In [20]:
firstrow[1]

'chocolate chip'

In [21]:
firstrow.cookie_name

'chocolate chip'

In [22]:
rp = cnn.execute(s)
for r in rp:
    print(r.cookie_name)

2025-05-16 08:43:11,520 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-16 08:43:11,522 INFO sqlalchemy.engine.Engine [cached since 0.5436s ago] ()
chocolate chip
peanut butter
oatmeal raisin
peanut butter
oatmeal raisin


In [23]:
rp = cnn.execute(s)
rp.first()

2025-05-16 08:43:11,629 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-16 08:43:11,630 INFO sqlalchemy.engine.Engine [cached since 0.651s ago] ()


(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50'))

In [24]:
rp = cnn.execute(s)
rp.fetchone()

2025-05-16 08:43:11,737 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-16 08:43:11,738 INFO sqlalchemy.engine.Engine [cached since 0.7597s ago] ()


(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50'))

In [25]:
rp = cnn.execute(s)
rp.scalar()

2025-05-16 08:43:11,845 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-16 08:43:11,846 INFO sqlalchemy.engine.Engine [cached since 0.8674s ago] ()


1

In [26]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
rp = cnn.execute(s)

2025-05-16 08:43:11,963 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies
2025-05-16 08:43:11,964 INFO sqlalchemy.engine.Engine [generated in 0.00122s] ()


In [27]:
rp.keys()

RMKeyView(['cookie_name', 'quantity'])

In [28]:
result = rp.first()
result

('chocolate chip', 252)

In [29]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity)
rp = cnn.execute(s)
for ck in rp:
    print(f"{ck.cookie_name}--{ck.quantity}")

2025-05-16 08:43:12,338 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies ORDER BY cookies.quantity
2025-05-16 08:43:12,339 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ()
peanut butter--24
peanut butter--24
oatmeal raisin--100
oatmeal raisin--100
chocolate chip--252


In [30]:
from sqlalchemy import desc

s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(desc(cookies.c.quantity))
rp = cnn.execute(s)
for ck in rp:
    print(ck)

2025-05-16 08:43:12,463 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies ORDER BY cookies.quantity DESC
2025-05-16 08:43:12,464 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ()
('chocolate chip', 252)
('oatmeal raisin', 100)
('oatmeal raisin', 100)
('peanut butter', 24)
('peanut butter', 24)


In [31]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity)
s = s.limit(3)
rp = cnn.execute(s)
print([r.cookie_name for r in rp])

2025-05-16 08:43:12,630 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies ORDER BY cookies.quantity
 LIMIT ? OFFSET ?
2025-05-16 08:43:12,631 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (3, 0)
['peanut butter', 'peanut butter', 'oatmeal raisin']


In [32]:
from sqlalchemy import func

s = select(func.sum(cookies.c.quantity))
rp = cnn.execute(s)
print(rp.scalar())

2025-05-16 08:43:12,755 INFO sqlalchemy.engine.Engine SELECT sum(cookies.quantity) AS sum_1 
FROM cookies
2025-05-16 08:43:12,756 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()
500


In [33]:
s = select(func.count(cookies.c.cookie_name))
rp = cnn.execute(s)
r = rp.first()
print(r)

2025-05-16 08:43:12,872 INFO sqlalchemy.engine.Engine SELECT count(cookies.cookie_name) AS count_1 
FROM cookies
2025-05-16 08:43:12,873 INFO sqlalchemy.engine.Engine [generated in 0.00112s] ()
(5,)


In [34]:
s = select(func.count(cookies.c.cookie_name).label("inv_cnt"))
rp = cnn.execute(s)
r = rp.first()
print(r)

2025-05-16 08:43:12,988 INFO sqlalchemy.engine.Engine SELECT count(cookies.cookie_name) AS inv_cnt 
FROM cookies
2025-05-16 08:43:12,990 INFO sqlalchemy.engine.Engine [generated in 0.00124s] ()
(5,)


In [35]:
r.inv_cnt

5

In [36]:
s = select(cookies).where(cookies.c.cookie_name == "chocolate chip")
rp = cnn.execute(s)
r = rp.first()
print(r)

2025-05-16 08:43:13,330 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
2025-05-16 08:43:13,331 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('chocolate chip',)
(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50'))


In [37]:
[n for n in dir(r) if not n.startswith("_")]

['count', 'index', 't', 'tuple']

In [38]:
r._tuple()

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50'))

In [39]:
s = select(cookies).where(cookies.c.cookie_name.like("%chocolate%"))
rp = cnn.execute(s)
for r in rp.fetchall():
    print(r)

2025-05-16 08:43:13,719 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies 
WHERE cookies.cookie_name LIKE ?
2025-05-16 08:43:13,719 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ('%chocolate%',)
(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 252, Decimal('0.50'))


In [40]:
s = select(cookies.c.cookie_name, "SKU- " + cookies.c.cookie_sku)
for row in cnn.execute(s):
    print(row)

2025-05-16 08:43:13,834 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, ? || cookies.cookie_sku AS anon_1 
FROM cookies
2025-05-16 08:43:13,835 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ('SKU- ',)
('chocolate chip', 'SKU- CC01')
('peanut butter', 'SKU- PB01')
('oatmeal raisin', 'SKU- EWW01')
('peanut butter', 'SKU- PB01')
('oatmeal raisin', 'SKU- EWW01')


In [41]:
from sqlalchemy import cast

s = select(
    cookies.c.cookie_name,
    cast((cookies.c.quantity * cookies.c.unit_cost), Numeric(12, 2)).label("inv_cost"),
)
for row in cnn.execute(s):
    print(f"{row.cookie_name}--{row.inv_cost}")

2025-05-16 08:43:13,941 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, CAST(cookies.quantity * cookies.unit_cost AS NUMERIC(12, 2)) AS inv_cost 
FROM cookies
2025-05-16 08:43:13,942 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()
chocolate chip--126.00
peanut butter--6.00
oatmeal raisin--100.00
peanut butter--6.00
oatmeal raisin--100.00


In [42]:
from sqlalchemy import and_, or_, not_

In [43]:
s = select(cookies).where(and_(cookies.c.quantity > 23, cookies.c.unit_cost < 0.4))
for row in cnn.execute(s):
    print(row.cookie_name)

2025-05-16 08:43:14,147 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies 
WHERE cookies.quantity > ? AND cookies.unit_cost < ?
2025-05-16 08:43:14,148 INFO sqlalchemy.engine.Engine [generated in 0.00136s] (23, 0.4)
peanut butter
peanut butter


In [44]:
s = select(cookies).where(
    or_(cookies.c.quantity.between(10, 50), cookies.c.cookie_name.contains("chip"))
)
for row in cnn.execute(s):
    print(row.cookie_name)

2025-05-16 08:43:14,247 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies 
WHERE cookies.quantity BETWEEN ? AND ? OR (cookies.cookie_name LIKE '%' || ? || '%')
2025-05-16 08:43:14,248 INFO sqlalchemy.engine.Engine [generated in 0.00131s] (10, 50, 'chip')
chocolate chip
peanut butter
peanut butter


In [46]:
from sqlalchemy import insert

ins = insert(line_items)
order_items = [
    {"order_id": 1, "cookie_id": 1, "quantity": 2, "extended_cost": 1.00},
    {"order_id": 1, "cookie_id": 3, "quantity": 12, "extended_cost": 3.00},
]
result = cnn.execute(ins, order_items)
ins = insert(orders).values(user_id=2, order_id=2)
result = cnn.execute(ins)
cnn.commit()

2025-05-16 09:23:30,158 INFO sqlalchemy.engine.Engine INSERT INTO line_items (order_id, cookie_id, quantity, extended_cost) VALUES (?, ?, ?, ?)
2025-05-16 09:23:30,159 INFO sqlalchemy.engine.Engine [generated in 0.00160s] [(1, 1, 2, 1.0), (1, 3, 12, 3.0)]
2025-05-16 09:23:30,162 INFO sqlalchemy.engine.Engine INSERT INTO orders (order_id, user_id, shipped) VALUES (?, ?, ?)
2025-05-16 09:23:30,162 INFO sqlalchemy.engine.Engine [cached since 148s ago] (2, 2, 0)
2025-05-16 09:23:30,164 INFO sqlalchemy.engine.Engine COMMIT


In [47]:
ins = insert(line_items)
order_items = [
    {
        "order_id": 2,
        "cookie_id": 1,
        "quantity": 24,
        "extended_cost": 12.0,
    },
    {"order_id": 2, "cookie_id": 4, "quantity": 6, "extended_cost": 6.0},
]
result = cnn.execute(ins, order_items)
cnn.commit()

2025-05-16 09:26:02,816 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-16 09:26:02,818 INFO sqlalchemy.engine.Engine INSERT INTO line_items (order_id, cookie_id, quantity, extended_cost) VALUES (?, ?, ?, ?)
2025-05-16 09:26:02,819 INFO sqlalchemy.engine.Engine [cached since 152.7s ago] [(2, 1, 24, 12.0), (2, 4, 6, 6.0)]
2025-05-16 09:26:02,821 INFO sqlalchemy.engine.Engine COMMIT


In [48]:
cookiemon_orders = select(
    orders.c.order_id,
    users.c.username,
    users.c.phone,
    cookies.c.cookie_name,
    line_items.c.quantity,
    line_items.c.extended_cost,
)
cookiemon_orders = cookiemon_orders.select_from(
    orders.join(users).join(line_items).join(cookies)
).where(users.c.username == "cookiemon")

result = cnn.execute(cookiemon_orders).fetchall()
for row in result:
    print(row)

2025-05-16 11:48:19,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-16 11:48:19,921 INFO sqlalchemy.engine.Engine SELECT orders.order_id, users.username, users.phone, cookies.cookie_name, line_items.quantity, line_items.extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = ?
2025-05-16 11:48:19,922 INFO sqlalchemy.engine.Engine [generated in 0.00253s] ('cookiemon',)
(1, 'cookiemon', '111-111-1111', 'chocolate chip', 2, Decimal('1.00'))
(1, 'cookiemon', '111-111-1111', 'oatmeal raisin', 12, Decimal('3.00'))


In [50]:
str(orders.join(line_items).join(cookies))

'orders JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id'

In [52]:
str(users.outerjoin(orders))

'users LEFT OUTER JOIN orders ON users.user_id = orders.user_id'

In [53]:
all_orders = select(users.c.username, func.count(orders.c.order_id))
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)
result = cnn.execute(all_orders).fetchall()
for row in result:
    print(row)

2025-05-16 11:53:54,427 INFO sqlalchemy.engine.Engine SELECT users.username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username
2025-05-16 11:53:54,428 INFO sqlalchemy.engine.Engine [cached since 1.134e+04s ago] ()
('cakeeater', 1)
('cookiemon', 1)
('pieguy', 0)


In [55]:
em_table = Table(
    "employee",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("manager", None, ForeignKey("employee.id")),
    Column("name", String(255)),
    extend_existing=True,
)

In [56]:
manager = em_table.alias("mgr")
stmt = select(
    em_table.c.name, and_(employee_table.c.id == manager.c.id, manager.c.name == "Fred")
)
print(stmt)

SELECT employee.name, employee.id = mgr.id AND mgr.name = :name_1 AS anon_1 
FROM employee, employee AS mgr


In [57]:
manager = employee_table.alias()
stmt = select(
    em_table.c.name, and_(em_table.c.id == manager.c.id, manager.c.name == "Fred")
)
print(stmt)

SELECT employee.name, employee.id = employee_1.id AND employee_1.name = :name_1 AS anon_1 
FROM employee, employee AS employee_1


In [60]:
all_orders = select(users.c.username, func.count(orders.c.order_id))
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)
result = cnn.execute(all_orders).fetchall()
for row in result:
    print(row)

2025-05-16 14:39:57,970 INFO sqlalchemy.engine.Engine SELECT users.username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username
2025-05-16 14:39:57,971 INFO sqlalchemy.engine.Engine [cached since 2.111e+04s ago] ()
('cakeeater', 1)
('cookiemon', 1)
('pieguy', 0)


In [61]:
def get_orders_by_customer(cust_name):
    cust_orders = select(
        orders.c.order_id,
        users.c.username,
        users.c.phone,
        cookies.c.cookie_name,
        line_items.c.quantity,
        line_items.c.extended_cost,
    )
    cust_orders = cust_orders.select_from(
        users.join(orders).join(line_items).join(cookies)
    )
    cust_orders = cust_orders.where(users.c.username == cust_name)
    result = cnn.execute(cust_orders).fetchall()
    return result


get_order_by_customer("cakeeater")

2025-05-16 14:43:52,896 INFO sqlalchemy.engine.Engine SELECT orders.order_id, users.username, users.phone, cookies.cookie_name, line_items.quantity, line_items.extended_cost 
FROM users JOIN orders ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = ?
2025-05-16 14:43:52,897 INFO sqlalchemy.engine.Engine [cached since 2.134e+04s ago] ('cakeeater',)


[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, Decimal('12.00'))]

In [62]:
cols = [orders.c.order_id, users.c.username, users.c.phone]
cols

[Column('order_id', Integer(), table=<orders>, primary_key=True, nullable=False),
 Column('username', String(length=15), table=<users>, nullable=False),
 Column('phone', String(length=20), table=<users>, nullable=False)]

In [ ]:
def get_order_by_customer(cust_name,shipped=None,details=False):
    if details:
        cust_orders=select(
            orders.c.order_id,users.c['username','phone'],
            line_items.c.extended_cost,cookies.c.cookie_name,line_items.c.quantity
        )
        joins=users.join()